<center><img src="./imge/GLODON.png" alt="广联达" style="width: 500px;"/></center>

## AI劳务人脸识别

@  公司名称：广联达股份有限公司<br>
@  文件名：3.AI劳务人脸识别.ipynb <br>
@  版本：V1.0 <br>
@  author: glodon.com<br>
@  说明：学习AI人脸识别的原理过程<br>

本节内容主要为基于Dlib来实现人脸身份识别；

1.安装依赖包

sudo apt-get update

sudo apt-get install cmake libopenblas-dev liblapack-dev libjpeg-dev

2.下载dlib之前，需要一个交换文件，由于nano的内存只有4G，编译过程中需要用到交换文件；

git clone https://github.com/JetsonHacksNano/installSwapfile

cd installSwapfile

./installSwapfile.sh

3.下载Dlib安装包，解压

wget http://dlib.net/files/dlib-19.17.tar.bz2

tar jxvf dlib-19.17.tar.bz2

cd dlib-19.17

4.需要对源代码中的一行代码进行修改，使得适配于Jetson Nano

找到如下目录：/dlib-19.17/dlib/cuda/cudnn_dlibapi.cpp

注释下面代码：

//forward_algo = forward_best_algo;

5.安装dlib

cd ..

cd dlib-19.17

sudo python3 setup.py install

6.安装face_recognition库

sudo pip3 install face_recognition

## 1.人脸识别库-检测人脸

In [13]:
import face_recognition
import cv2
print(cv2.__version__)

3.2.0


In [14]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [15]:
# 导入显示控件的依赖库
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# 创建显示原始图片的控件
origin_imge = widgets.Image(format='jpeg', width=640, height=480)
display(origin_imge)

Image(value=b'', format='jpeg', height='480', width='640')

In [16]:
# 创建显示检测结果的控件
face_imge = widgets.Image(format='jpeg', width=640, height=480)
display(face_imge)

Image(value=b'', format='jpeg', height='480', width='640')

In [17]:
# 输入图片路径，这里可替换成自己的图片目录路径，使用绝对路径即可
# 如：/home/eternity/Desktop/test/*.jpg
image=face_recognition.load_image_file('./images/face/unknown/u14.png')
origin_imge.value = bgr8_to_jpeg(cv2.cvtColor(image,cv2.COLOR_RGB2BGR))
face_locations=face_recognition.face_locations(image)
print(face_locations)
image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
for(row1,col1,row2,col2) in face_locations:
    # 画框
    cv2.rectangle(image,(col1,row1),(col2,row2),(0,0,255),2)
face_imge.value = bgr8_to_jpeg(image)

[(136, 225, 226, 135)]


## 2.人脸身份识别

In [18]:
import face_recognition
import cv2
print(cv2.__version__)

3.2.0


In [19]:
# 导入显示控件的依赖库
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# 创建显示添加的第一张人脸库图片的控件区域
feature_imge1 = widgets.Image(format='jpeg', width=640, height=480)
display(feature_imge1)

Image(value=b'', format='jpeg', height='480', width='640')

In [20]:
# 创建显示添加的第二张人脸库图片的控件区域
feature_imge2 = widgets.Image(format='jpeg', width=640, height=480)
display(feature_imge2)

Image(value=b'', format='jpeg', height='480', width='640')

In [21]:
# 创建显示添加的第三张人脸库图片的控件区域
feature_imge3 = widgets.Image(format='jpeg', width=640, height=480)
display(feature_imge3)

Image(value=b'', format='jpeg', height='480', width='640')

In [22]:
# 创建显示添加的第四张人脸库图片的控件区域
feature_imge4 = widgets.Image(format='jpeg', width=640, height=480)
display(feature_imge4)

Image(value=b'', format='jpeg', height='480', width='640')

In [23]:
# 创建显示待识别人脸图片的控件区域
query_imge = widgets.Image(format='jpeg', width=640, height=480)
display(query_imge)

Image(value=b'', format='jpeg', height='480', width='640')

In [24]:
#自己定义自己的人脸库时，需要建立文件，添加一定数量的人脸数据，作为人脸底库
#下方代码中，添加了4个人的人脸数据，作为底库
# 注意：下面代码中涉及到的主要处理函数，如face_recognition.face_encodings，face_recognition.compare_faces等，均是调用了
# face_recognition识别库，具体有关函数的说明可查看face_recognition官方说明，https://github.com/ageitgey/face_recognition；

# 特征库图片路径，这里可替换成自己的图片目录路径，使用绝对路径即可
# 如：/home/eternity/Desktop/test/*.jpg
donFace=face_recognition.load_image_file('./images/face/known/Donald Trump.jpg')# 输入图片路径，这里可替换成自己的图片目录路径，使用绝对路径即可
feature_imge1.value = bgr8_to_jpeg(cv2.cvtColor(donFace,cv2.COLOR_RGB2BGR))
donEncode = face_recognition.face_encodings(donFace)[0]

nancyFace=face_recognition.load_image_file('./images/face/known/Nancy Pelosi.jpg')# 输入图片路径，这里可替换成自己的图片目录路径，使用绝对路径即可
feature_imge2.value = bgr8_to_jpeg(cv2.cvtColor(nancyFace,cv2.COLOR_RGB2BGR))
nancyEncode = face_recognition.face_encodings(nancyFace)[0]

penceFace=face_recognition.load_image_file('./images/face/known/Mike Pence.jpg')# 输入图片路径，这里可替换成自己的图片目录路径，使用绝对路径即可
feature_imge3.value = bgr8_to_jpeg(cv2.cvtColor(penceFace,cv2.COLOR_RGB2BGR))
penceEncode = face_recognition.face_encodings(penceFace)[0]

weijunFace=face_recognition.load_image_file('./images/face/known/sunweijun.jpg')# 输入图片路径，这里可替换成自己的图片目录路径，使用绝对路径即可
feature_imge4.value = bgr8_to_jpeg(cv2.cvtColor(weijunFace,cv2.COLOR_RGB2BGR))
weijunEncode = face_recognition.face_encodings(weijunFace)[0]

Encodings=[donEncode,nancyEncode,penceEncode,weijunEncode]
Names=['The Donald','Nancy Pelosi','Mike Pence','sunweijun'] # 人脸库中四个人的名称

font = cv2.FONT_HERSHEY_SIMPLEX
# 待匹配的未知人脸
testImage=face_recognition.load_image_file('./images/face/unknown/u16.png')# 输入图片路径，这里可替换成自己的图片目录路径，使用绝对路径即可
# 如：/home/eternity/Desktop/test/*.jpg

facePositions=face_recognition.face_locations(testImage)
allEncodings=face_recognition.face_encodings(testImage,facePositions)

testImage=cv2.cvtColor(testImage,cv2.COLOR_RGB2BGR)
query_imge.value = bgr8_to_jpeg(testImage)

In [25]:
# 创建显示人脸匹配结果的控件区域
face_imge1 = widgets.Image(format='jpeg', width=640, height=480)
display(face_imge1)

Image(value=b'', format='jpeg', height='480', width='640')

In [26]:
for (top,right,bottom,left),face_encoding in zip(facePositions,allEncodings):
    name='Unknown Person'
    matches=face_recognition.compare_faces(Encodings,face_encoding)
    if True in matches:
        first_match_index=matches.index(True)
        name=Names[first_match_index]
    # 画框
    cv2.rectangle(testImage,(left,top),(right,bottom),(0,0,255),2)
    # 写人脸对应的name到图片
    cv2.putText(testImage,name,(left,top-6),font,.75,(0,255,255),2)
face_imge1.value = bgr8_to_jpeg(testImage)

## 3.通过摄像头进行面部身份识别


### 该部分实验内容注意事项：

1.一个cell一个cell的执行，不可run all，因为后面会涉及到关闭摄像头的程序代码；

2.请按照最后一个cell中的提示，在结束退出实验时，请务必正确关闭摄像头，切记！！！

In [1]:
import face_recognition
import cv2
import os
print(cv2.__version__)

3.2.0


In [1]:
# 导入显示控件的依赖库
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# 创建显示特征库人脸的控件区域
feature_imge = widgets.Image(format='jpeg', width=640, height=480)
display(feature_imge)

Image(value=b'', format='jpeg', height='480', width='640')

In [3]:
# Extract feature,and Build face feature database
Encodings=[]
Names=[]

# 在识别人脸之前，需要先提取人脸库特征
# 为了教学快速体验，这里只添加了2张人脸到人脸库，可以自定义数量
image_dir='./images/face/known2' # 这里是人脸特征底库的图片路径，同样可以自定义自己的特征库，可以添加自己的人脸数据，替换成自己的绝对路径即可
# 如：/home/eternity/Desktop/test/
for root,dirs,files in os.walk(image_dir):
    print(files)
    for file in files:
        path = os.path.join(root,file)
        print(path)
        name=os.path.splitext(file)[0]
        print(name)
        person=face_recognition.load_image_file(path)
        feature_imge.value = bgr8_to_jpeg(cv2.cvtColor(person,cv2.COLOR_RGB2BGR))
        encoding=face_recognition.face_encodings(person)[0]
        Encodings.append(encoding)
        Names.append(name)
print(Names)

font=cv2.FONT_HERSHEY_SIMPLEX

['sunweijun.jpg', 'zhanghan.jpg']
./images/face/known2/sunweijun.jpg
sunweijun
./images/face/known2/zhanghan.jpg
zhanghan
['sunweijun', 'zhanghan']


In [4]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)


In [11]:
dispW=480
dispH=320
flip=4
#Uncomment These next Two Line for Pi Camera
camSet='nvarguscamerasrc !  video/x-raw(memory:NVMM), width=3264, height=2464, format=NV12, framerate=21/1 ! nvvidconv flip-method='+str(flip)+' ! video/x-raw, width='+str(dispW)+', height='+str(dispH)+', format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink'
cap= cv2.VideoCapture(camSet)

NameError: name 'cv2' is not defined

In [2]:
# 创建显示识别结果的控件区域
face_imge2 = widgets.Image(format='jpeg', width=640, height=480)
display(face_imge2)


Image(value=b'', format='jpeg', height='480', width='640')

In [3]:
def Video_display():
    while True:      
        ret, img = cap.read()
        testImage = img.copy()  
        facePositions=face_recognition.face_locations(testImage)
        allEncodings=face_recognition.face_encodings(testImage,facePositions)       
        for (top,right,bottom,left),face_encoding in zip(facePositions,allEncodings):
            name='Unknown Person'
            matches=face_recognition.compare_faces(Encodings,face_encoding)
            if True in matches:
                first_match_index=matches.index(True)
                name=Names[first_match_index]
            # 画框
            cv2.rectangle(testImage,(left,top),(right,bottom),(0,0,255),2)
            # 写人脸对应的name到图片
            cv2.putText(testImage,name,(left,top-6),font,.75,(0,255,255),2)
        face_imge2.value = bgr8_to_jpeg(testImage)

    cap.release()   

In [11]:
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-423e7c42211e>", line 4, in Video_display
    testImage = img.copy()
AttributeError: 'NoneType' object has no attribute 'copy'



In [12]:
# 结束线程任务
stop_thread(t)

ValueError: invalid thread id